# Convert raw txt output to JSON

Columns in txt are:

```bifurcation ID, number of bounces, pr, ptheta, pphi, energy, rocking number, in phase/out of phase, period```

In [1]:
import math

# Returns a set of all prime factors of n
def primeFactors(n):
    if (n == 0):
        return set([])
    factors = []

    # Print the number of twos that divide n 
    while n % 2 == 0:
        factors.append(2)
        n = n // 2
          
    # n must be odd at this point 
    # so a skip of 2 (i = i + 2) can be used 
    for i in range(3, int(math.sqrt(n))+1, 2): 
        # while i divides n, print i and divide n
        while n % i == 0:
            factors.append(i)
            n = n // i
              
    # Condition if n is a prime
    # number greater than 2
    if n > 2:
        factors.append(n)
    
    return set(factors)

# Returns a set of all factors of n
def factors(n):
    if (n == 0):
        return set([])
    
    factors = []
    for i in range(1, n//2+1):
        if n % i == 0:
            factors.append(i)
    return set(factors)

print(primeFactors(0))
print(primeFactors(98))
print(factors(98))

print(primeFactors(74))
print(factors(74))

set()
{2, 7}
{1, 2, 7, 14, 49}
{2, 37}
{1, 2, 37}


In [40]:
a = {3:2, 5:4}
print(5 in a)

True


# Test code for something else

In [ ]:
class BifLoc:
    def __init__(self, E, m):
        self.E = E
        self.m = m

# Does a binary search to find where E belongs in the bifurcation locations.
# If the region is surrounded by two found bifurcations such that the
# rocking numbers are next to each other, then the region is empty.
# m_max is the maximum rocking number we're going to allow.
def is_empty(E, bif_locs, m_max):
    if E == 0:
        return True
    i = 0
    j = len(bif_locs)
    while j-i > 1:
        k = (i+j)//2
        if bif_locs[k].E <= E:
            i = k
        else:
            j = k
    if bif_locs[i].m == bif_locs[j].m-1 or bif_locs[i].m == m_max:
        return True
    return False

bif_locs = [BifLoc(-1/3, 0), BifLoc(-.22, 3), BifLoc(-.1, 4), BifLoc(0, math.inf)]
bif_locs[-1].E
print(is_empty(-.2, bif_locs, 9) == True)
print(is_empty(-.23, bif_locs, 9) == False)
print(is_empty(-.22, bif_locs, 9) == True)
print(is_empty(-.04, bif_locs, 9) == False)
print(is_empty(-1/3, bif_locs, 9) == False)
print(is_empty(0, bif_locs, 9) == True)

print(is_empty(-.2, bif_locs, 4) == True)
print(is_empty(-.23, bif_locs, 4) == False)
print(is_empty(-.22, bif_locs, 4) == True)
print(is_empty(-.04, bif_locs, 4) == True)
print(is_empty(-1/3, bif_locs, 4) == False)
print(is_empty(0, bif_locs, 4) == True)

print(is_empty(-.2, bif_locs, 5) == True)
print(is_empty(-.23, bif_locs, 5) == False)
print(is_empty(-.22, bif_locs, 5) == True)
print(is_empty(-.04, bif_locs, 5) == False)
print(is_empty(-1/3, bif_locs, 5) == False)
print(is_empty(0, bif_locs, 5) == True)


In [125]:
import queue 
  
# From class queue, Queue is 
# created as an object Now L 
# is Queue of a maximum  
# capacity of 20 
L = queue.Queue() 
  
# Data is inserted into Queue 
# using put() Data is inserted 
# at the end 
L.put(5) 
L.put(9) 
L.put(1) 
L.put(7) 
  
# get() takes data out from 
# the Queue from the head  
# of the Queue 
print(L.get()) 
print(L.get()) 
print(L.get()) 
print(L.get()) 

5
9
1
7


# Simple conversion not checking for prime factors

Use this one!



In [11]:
import json
import re
import os
from os import listdir
from os.path import isfile, join

# def getRockingNumber(s):
#     if s['phase'] == 1:
#         # In-phase
#         return s['theta_crossings']
#     else:
#         # Out-of-phase
#         return s['pphi_rocking']

def convert():
    data = []
        
    mypath = './states'
    files = [mypath+'/'+f for f in listdir(mypath) if isfile(join(mypath, f)) and os.path.splitext(f)[1] == '.txt']
    files = sorted(files)
#     for num_bounces in range(1, 1000):
#         fn = './states{:03d}.txt'.format(num_bounces)
#     files = [mypath+'/states001.txt']
    for fn in files:
        exists = os.path.isfile(fn)
        if exists:
            with open(fn, 'r') as infile:
                print('reading {}'.format(fn))
                cur_mode = -1
                lines = [line for line in infile]
                for i, line in enumerate(lines):
                    line = line.strip()
                    (mode_id, n, pr, ptheta, pphi, E, ptheta_rocking, pphi_rocking, phase, period,
                    theta_crossings, phi_crossings, beta_crossings) = line.split(' ')
                    is_bifurcation = mode_id != cur_mode
                    state = { 'id':int(mode_id), 'bifurcation':is_bifurcation, 'n':int(n), 'pr':float(pr),
                             'ptheta':float(ptheta), 'pphi':float(pphi), 'E':float(E),
                             'ptheta_rocking':int(ptheta_rocking), 'pphi_rocking':int(pphi_rocking),
                             'phase':int(phase), 'period':float(period), 'theta_crossings':int(theta_crossings),
                             'phi_crossings':int(phi_crossings), 'beta_crossings':int(beta_crossings) }
#                     is_valid = (float(pphi) > 1e-4)
                    is_valid = True
                    if is_bifurcation:
                        # Don't include modes with only a bifurcation state.
                        is_valid = i < len(lines) - 1
                        if is_valid:
                            (next_mode_id,_,_,_,_,_,_,_,_,_,_,_,_) = lines[i+1].split(' ')
                            is_valid = (next_mode_id == mode_id)
                    if is_valid:
                        data.append(state)
                    cur_mode = mode_id

    with open('./states_all.json', 'w') as outfile:  
        json.dump(data, outfile)

    print('done')

convert()
# %timeit convert()

reading ./states/states001.txt
reading ./states/states001_8_0.txt
reading ./states/states002.txt
reading ./states/states002_8_0.txt
reading ./states/states003.txt
reading ./states/states004.txt
reading ./states/states005.txt
reading ./states/states006.txt
reading ./states/states007.txt
reading ./states/states009.txt
reading ./states/states009_198_126.txt
reading ./states/states011.txt
reading ./states/states013.txt
reading ./states/states015.txt
reading ./states/states019.txt
reading ./states/states023.txt
reading ./states/states027.txt
reading ./states/states027_166_145.txt
reading ./states/states027_187_155.txt
reading ./states/states031.txt
reading ./states/states039.txt
reading ./states/states047.txt
reading ./states/states047_262_244.txt
reading ./states/states047_302_286.txt
reading ./states/states047_334_331.txt
reading ./states/states055.txt
reading ./states/states055_242_226.txt
reading ./states/states055_278_263.txt
reading ./states/states055_305_298.txt
reading ./states/stat

# Check for factors

In [ ]:
import json
import re

def convert():
    state_map = {}
#     pfactors = [primeFactors(i) for i in range(129)]
    pfactors = [factors(i) for i in range(129)]
    cur_mode = -1
    skip_mode = False
    data = []
    with open('./states.txt', 'r') as infile:
        for line in infile:
            line = line.strip()
            (mode_id, n, pr, ptheta, pphi, E, rocking, phase, period) = line.split(' ')
            mode_id = int(mode_id)
            n = int(n)
            E = float(E)
            rocking = int(rocking)
            state = { 'id':int(mode_id), 'n':int(n), 'pr':float(pr),
                         'ptheta':float(ptheta), 'pphi':float(pphi), 'E':float(E),
                         'rocking':int(rocking), 'phase':int(phase), 'period':float(period) }
            
            if mode_id != cur_mode:
                # Look to see if a prime factor of n already added this state. This functionality relies on
                # the text data coming to being sorted on n.
                unique = True
                for factor_n in pfactors[n]:
                    if (unique and (rocking//(n//factor_n), factor_n) in state_map):
                        # A prime factor bouncing number has a mode with this rocking number.
                        # Check to see if it is the same mode.
                        prev_state = state_map[(rocking//(n//factor_n), factor_n)]
                        if abs(prev_state['E'] - state['E']) < 0.001:
                            # The current state is redundant. Ignore.
                            unique = False
                            print('Filtering out ({},{},{},{}) from ({},{},{},{})'.format(
                                prev_state['n'], prev_state['rocking'], prev_state['E'], prev_state['pphi'],
                                state['n'], state['rocking'], state['E'], state['pphi']))
                if unique:
                    state_map[(rocking, n)] = state
                    data.append(state)
                    skip_mode = False
                else:
                    skip_mode = True
            elif not skip_mode:
                data.append(state)
            
            cur_mode = mode_id

    with open('./states_unique.json', 'w') as outfile:  
        json.dump(data, outfile)

    print('done')

convert()
# %timeit convert()